In [6]:
from metagpt.roles.product_manager import ProductManager
from metagpt.actions import Action
import re
import os
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.logs import logger

In [1]:
# 可导入任何角色，初始化它，用一个开始的消息运行它，完成！

prompt = f"""
# Role：软件开发团队

## Background :

我是一个软件开发团队。
现在要用html、js、vue3、element-plus开发一个刷题程序。
刷题可以让人们对题目中涉及的知识点有更深的掌握。

## Profile:
- author: 黎伟
- version: 0.1
- language: 中文
- description: 我是一软件开发团队。

## Goals:
- 用html、js、vue3、element-plus开发一个刷题程序的开发需求文档。

## Constrains:
1. 最后交付的程序是一个html单文件，不要有其他任何文件。
2. 题目的题型至少包括两道判断题、两道选择题、两道填空题。
3. 题目的内容与人工智能的agent基本理论相关。
4. 刷题程序至少给出10道样例题目。
5. 题目用列表的形式写到html文件的script部分。
6. vue3、element-plus采用cdn的形式在html的header部分引入。

## Skills:
1. 具有强大的js语言开发能力
2. 熟悉vue3、element-plus的使用
3. 对人工智能的agent基本理论有较好理解
4. 拥有排版审美, 会利用序号, 缩进, 分隔线和换行符等等来美化信息排版


请结合上述要求完善刷题程序的开发需求文档。
"""
async def main():
    role = ProductManager()
    result = await role.run(prompt)
    
await main()

2025-02-17 11:42:31.076 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to c:\Users\zata\Desktop\wow\01-wow-agent\04
2025-02-17 11:42:39.886 | INFO     | metagpt.roles.role:_act:391 - Alice(Product Manager): to do PrepareDocuments(PrepareDocuments)
2025-02-17 11:42:40.395 | INFO     | metagpt.utils.file_repository:save:57 - save to: C:\Users\zata\Desktop\wow\01-wow-agent\04\workspace\20250217114239\docs\requirement.txt


In [4]:
class TangPoem(Action):
    PROMPT_TEMPLATE: str = """
    根据主题{msg}写一首五言绝句的唐诗。只返回生成诗的内容，不要有其他文字。
    """
    async def run(self, msg: str):
        prompt = self.PROMPT_TEMPLATE.format(msg = msg)
        rsp = await self._aask(prompt)
        return rsp
tangshi = TangPoem()
rst = await tangshi.run('写一首表达上班很辛苦的唐诗')
print(rst)

朝朝暮暮勤，筋骨渐疲惫。
尘世多艰辛，谁解其中味。

2025-02-17 11:44:50.695 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-flash not found in TOKEN_COSTS.



朝朝暮暮勤，筋骨渐疲惫。
尘世多艰辛，谁解其中味。


# 创建写代码智能体
在 MetaGPT 中，Role 类是智能体的逻辑抽象。一个 Role 能执行特定的 Action，拥有记忆、思考并采用各种策略行动。基本上，它充当一个将所有这些组件联系在一起的凝聚实体。目前，让我们只关注一个执行动作的智能体，并看看如何定义一个最简单的 Role。

在这个示例中，我们创建了一个 SimpleCoder，它能够根据人类的自然语言描述编写代码。步骤如下：

我们为其指定一个名称和配置文件。
我们使用 self._init_action 函数为其配备期望的动作 SimpleWriteCode。
我们覆盖 _act 函数，其中包含智能体具体行动逻辑。我们写入，我们的智能体将从最新的记忆中获取人类指令，运行配备的动作，MetaGPT将其作为待办事项 (self.rc.todo) 在幕后处理，最后返回一个完整的消息。

In [7]:
from metagpt.actions import Action

class SimpleWriteCode(Action):
    PROMPT_TEMPLATE: str = """
    Write a python function that can {instruction} and provide two runnnable test cases.
    Return ```python your_code_here ```with NO other texts,
    your code:
    """

    name: str = "SimpleWriteCode"

    async def run(self, instruction: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = SimpleWriteCode.parse_code(rsp)

        return code_text

    @staticmethod
    def parse_code(rsp):
        pattern = r"```python(.*)```"
        match = re.search(pattern, rsp, re.DOTALL)
        code_text = match.group(1) if match else rsp
        return code_text




class SimpleCoder(Role):
    name: str = "Alice"
    profile: str = "SimpleCoder"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([SimpleWriteCode])

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        todo = self.rc.todo  # todo will be SimpleWriteCode()

        msg = self.get_memories(k=1)[0]  # find the most recent messages
        code_text = await todo.run(msg.content)
        msg = Message(content=code_text, role=self.profile, cause_by=type(todo))

        return msg

# 运行

async def main():
    msg = "write a function that calculates the sum of a list"
    role = SimpleCoder()
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)
    return result
    
        
rtn = await main()

2025-02-17 11:48:37.800 | INFO     | __main__:main:54 - write a function that calculates the sum of a list
2025-02-17 11:48:37.805 | INFO     | __main__:_act:40 - Alice(SimpleCoder): to do SimpleWriteCode(SimpleWriteCode)


```python
def sum_of_list(lst):
    return sum(lst)

# Test cases
print(sum_of_list([1, 2, 3, 4]))  # Should return 10
print(sum_of_list([5, 10, 15]))   # Should return 30
```

2025-02-17 11:48:39.407 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-flash not found in TOKEN_COSTS.
2025-02-17 11:48:39.416 | INFO     | __main__:main:56 - SimpleCoder: 
def sum_of_list(lst):
    return sum(lst)

# Test cases
print(sum_of_list([1, 2, 3, 4]))  # Should return 10
print(sum_of_list([5, 10, 15]))   # Should return 30

